# Demonstration of the Temperature Problem

### We'll start off by importing what we need

In [2]:
# The usual suspects...
import numpy as np
import numpy.linalg as npla

# We'll need to import these to deal with SPARSE matrices
import scipy
import scipy.sparse.linalg as spla
from scipy import sparse
from scipy import linalg

# These are so we can make pretty 2D & 3D plots...
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d
%matplotlib tk

## We'll create 3 functions for our use:
### 1. make_A(k)    --- creates the nxn matrix we need from a kxk sized room model
### 2. make_b(k)    --- creates the right-hand side vector in the Ax = b setup
### 3. radiator(k)    --- creates a radiator wall at a set temperature (default is 100)

In [8]:
def make_A(k):
    """Create the matrix for the temperature problem on a k-by-k grid (2-dimensional).
    Parameters: 
      k: number of grid points in each dimension.
    Outputs:
      A: the sparse k**2-by-k**2 matrix representing the finite difference approximation to Poisson's equation.
    """
    # First make a list with one triple (row, column, value) for each nonzero element of A
    triples = []
    for i in range(k):
        for j in range(k):
            # what row of the matrix is grid point (i,j)?
            row = j + i*k   # this is the row index of A[i,j]
            # the diagonal element in this row
            triples.append((row, row, 4.0))
            # connect to left grid neighbor
            if j > 0:
                triples.append((row, row - 1, -1.0))
            # ... right neighbor
            if j < k - 1:
                triples.append((row, row + 1, -1.0))
            # ... neighbor above
            if i > 0:
                triples.append((row, row - k, -1.0))
            # ... neighbor below
            if i < k - 1:
                triples.append((row, row + k, -1.0))
        
    
    # Finally convert the list of triples to a scipy sparse matrix
    ndim = k*k
    rownum = [t[0] for t in triples]
    colnum = [t[1] for t in triples]
    values = [t[2] for t in triples]
    A = sparse.csr_matrix((values, (rownum, colnum)), shape = (ndim, ndim))
    
    return A 

In [9]:
def make_b(k, top = 0, bottom = 0, left = 0, right = 0):
    """Create the right-hand side for the temperature problem on a k-by-k grid.
    Parameters: 
      k: number of grid points in each dimension.
      top: list of k values for top boundary (optional, defaults to 0)
      bottom: list of k values for bottom boundary (optional, defaults to 0)
      left: list of k values for top boundary (optional, defaults to 0)
      right: list of k values for top boundary (optional, defaults to 0)
    Outputs:
      b: the k**2 element vector (as a numpy array) for the rhs of the Poisson equation with given boundary conditions
    """
    # Start with a vector of zeros
    ndim = k*k
    b = np.zeros(shape = ndim)
    
    # Fill in the four boundaries as appropriate
    b[0        : k       ] += top
    b[ndim - k : ndim    ] += bottom
    b[0        : ndim : k] += left   # each time we skip k elements
    b[k-1      : ndim : k] += right   
  
    return b

In [10]:
def radiator(k, width = .2, temperature = 100.):
    """Create one wall with a radiator
    Parameters: 
      k: number of grid points in each dimension; length of the wall.
      width: width of the radiator as a fraction of length of the wall (defaults to 0.2)
      temperature: temperature of the radiator (defaults to 100)
    Outputs:
      wall: the k element vector (as a numpy array) for the boundary conditions at the wall
    """
    rad_start = int(k * (0.5 - width/2))
    rad_end = int(k * (0.5 + width/2))
    wall = np.zeros(k)
    wall[rad_start : rad_end] = temperature
    
    return wall

## We will now set up the problem!

In [11]:
###########################
# Dimension of the square room (NOT the matrix)
# TRY using various values between 2 and 100
k = 3

###########################
# Set up the width of the radiator
# Default (per the function def) is 0.2
# This is a number relative to the length of the side
rk = radiator(k, width = 0.5)

###########################
# Create the matrix!
# A will be the sparse nxn matrix (where n = k**2)
A = make_A(k)
# convert A to matrix format
A_m = A.toarray()
###########################
# Create the b matrix: defines where the radiators go in the room
# SIDE=rk is placement of radiator, where SIDE can be (top, right, bottom, left)
b = make_b(k, right=rk)
#b = make_b(k, right=rk, left=rk, top=rk, bottom=rk)

###########################
# Print stuff to 's see what we got...
print("A_m: \n", A_m)
print("A_4: \n", make_A(2).toarray())
print("A:",A)
#print()
print(A.todense())
print("\nb:", b)
print("\nrad:", rk)

A_m: 
 [[ 4. -1.  0. -1.  0.  0.  0.  0.  0.]
 [-1.  4. -1.  0. -1.  0.  0.  0.  0.]
 [ 0. -1.  4.  0.  0. -1.  0.  0.  0.]
 [-1.  0.  0.  4. -1.  0. -1.  0.  0.]
 [ 0. -1.  0. -1.  4. -1.  0. -1.  0.]
 [ 0.  0. -1.  0. -1.  4.  0.  0. -1.]
 [ 0.  0.  0. -1.  0.  0.  4. -1.  0.]
 [ 0.  0.  0.  0. -1.  0. -1.  4. -1.]
 [ 0.  0.  0.  0.  0. -1.  0. -1.  4.]]
A_4: 
 [[ 4. -1. -1.  0.]
 [-1.  4.  0. -1.]
 [-1.  0.  4. -1.]
 [ 0. -1. -1.  4.]]
A:   (0, 0)	4.0
  (0, 1)	-1.0
  (0, 3)	-1.0
  (1, 0)	-1.0
  (1, 1)	4.0
  (1, 2)	-1.0
  (1, 4)	-1.0
  (2, 1)	-1.0
  (2, 2)	4.0
  (2, 5)	-1.0
  (3, 0)	-1.0
  (3, 3)	4.0
  (3, 4)	-1.0
  (3, 6)	-1.0
  (4, 1)	-1.0
  (4, 3)	-1.0
  (4, 4)	4.0
  (4, 5)	-1.0
  (4, 7)	-1.0
  (5, 2)	-1.0
  (5, 4)	-1.0
  (5, 5)	4.0
  (5, 8)	-1.0
  (6, 3)	-1.0
  (6, 6)	4.0
  (6, 7)	-1.0
  (7, 4)	-1.0
  (7, 6)	-1.0
  (7, 7)	4.0
  (7, 8)	-1.0
  (8, 5)	-1.0
  (8, 7)	-1.0
  (8, 8)	4.0
[[ 4. -1.  0. -1.  0.  0.  0.  0.  0.]
 [-1.  4. -1.  0. -1.  0.  0.  0.  0.]
 [ 0. -1.  4.  0.  0. -

In [12]:
# NOW!!! WE SOLVE IT!!
# We're going to use a different function (not .solve) altogether to solve Ax=b using a sparse matrix:
t = spla.spsolve(A,b)

# Reshape it back to a model (square) of the room
T = t.reshape(k,k)

print(t,"\n")
print(T)
# Looking at the data printed out is hopeless... you NEED to visualize it...

[ 5.80357143 16.07142857 39.73214286  7.14285714 18.75       42.85714286
  4.01785714  8.92857143 12.94642857] 

[[ 5.80357143 16.07142857 39.73214286]
 [ 7.14285714 18.75       42.85714286]
 [ 4.01785714  8.92857143 12.94642857]]


## Here come the Graphs!

**Let's start with a simple demonstration...**

### Ok... now for the real thing...

In [13]:
# Prep the plotter...
# Plot just a 2D image

#%matplotlib tk
plt.figure()
plt.imshow(T, cmap=cm.hot)    # T is the temperature matrix

### Let's try and project the 2D graph onto a 3D representation!

In [14]:
print(k)

3


In [15]:
# Prep the plotter again...
# X, Y are matrices of the x, y coordinates of each grid point  
k=3
X, Y = np.meshgrid(range(k), range(k))  
# %matplotlib tk
print(X,"\n")
print(Y)

[[0 1 2]
 [0 1 2]
 [0 1 2]] 

[[0 0 0]
 [1 1 1]
 [2 2 2]]


In [16]:
# Plot the 2D image as a 3D projection
fig = plt.figure()
ax = fig.gca(projection='3d')
ax = fig.gca()
# X is the x coordinate of each grid point, Y is the y coordinate of each grid point, T is the temperature of each grid point
ax.plot_surface(X, Y, T, cmap=cm.hot)

/var/folders/zr/26bbg2v930b01ykq1cqgjyb40000gn/T/ipykernel_96718/3855996379.py:3: MatplotlibDeprecationWarning: Calling gca() with keyword arguments was deprecated in Matplotlib 3.4. Starting two minor releases later, gca() will take no keyword arguments. The gca() function should only be used to get the current axes, or if no axes exist, create new axes with default keyword arguments. To create a new axes with non-default arguments, use plt.axes() or plt.subplot().
  ax = fig.gca(projection='3d')
